In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from io import StringIO
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
import glob

In [2]:
start_time = datetime.now()

In [3]:
dir = 'Y:\delete\California Office of Environmental Health Hazard Assessment'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [4]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\California Office of Environmental Health Hazard Assessment"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://oehha.ca.gov/proposition-65/proposition-65-list")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value='//*[@id="block-system-main"]/div/div/div/article[2]/div/span/a').click()
time.sleep(10)
driver.close()

In [13]:
path = 'Y:\\delete\\California Office of Environmental Health Hazard Assessment'
filenames = glob.glob(path + "\*.xlsx")
for f in filenames:
    dataset = pd.read_excel(f,skiprows =11)
dataset = dataset[['CAS No.']]
dataset = dataset[dataset['CAS No.'].notna()]
dataset.rename(columns = {'CAS No.':'CAS Number'}, inplace = True)

dataset.reset_index(inplace = True, drop = True)

In [14]:
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Not on Proposition 65 List'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [17]:
result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.head(2)

,CAS Number,value
0,26148-68-5,NaN
1,154229-18-2,NaN


In [20]:
result['Jurisdiction'] = 'US - California'
result['RegulatoryAgency'] = 'California Office of Environmental Health Hazard Assessment'
result['TypeOfRegulation'] = 'Health'
result['Criteria'] = 'Cancer, birth defects, reproductive harm'

result['Description'] = 'May cause cancer/reproductive toxicity'
result.loc[result['value'] == 'Not on Proposition 65 List', 'Description'] = 'Not known to cause cancer/reproductive toxicity'

result['HazardClassification'] = 'High'
result.loc[result['value'] == 'Not on Proposition 65 List', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = 'On Proposition 65 List'
result.loc[result['value'] == 'Not on Proposition 65 List', 'CauseOfClassification'] = 'Not on Proposition 65 List'

result['SourceName'] = 'California Office of Environmental Health Hazard Assessment'
result['SourceURL'] = 'https://oehha.ca.gov/proposition-65/proposition-65-list'

result = result.drop(['value'], axis=1)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,26148-68-5,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",May cause cancer/reproductive toxicity,High,On Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...
1,154229-18-2,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",May cause cancer/reproductive toxicity,High,On Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...
2,75-07-0,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",May cause cancer/reproductive toxicity,High,On Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...
3,60-35-5,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",May cause cancer/reproductive toxicity,High,On Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...
4,59-66-5,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",May cause cancer/reproductive toxicity,High,On Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...
...,...,...,...,...,...,...,...,...,...,...
968,12122-67-7,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",May cause cancer/reproductive toxicity,High,On Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...
969,14807-96-6,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",Not known to cause cancer/reproductive toxicity,Low,Not on Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...
970,7631-86-91,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",Not known to cause cancer/reproductive toxicity,Low,Not on Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...
971,9002-84-0,US - California,California Office of Environmental Health Haza...,Health,"Cancer, birth defects, reproductive harm",Not known to cause cancer/reproductive toxicity,Low,Not on Proposition 65 List,California Office of Environmental Health Haza...,https://oehha.ca.gov/proposition-65/propositio...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [5]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:00:35.379241
